In [12]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [(datetime.datetime.fromisoformat(res['time'][:19])).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).hour]) #時間を追加
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data




def select_data(year_from = 2010, month_from = 1, 
                year_to = 2020, month_to = 9,
                money='USD_JPY', foot = 'H1'):

    filename = str(year_from)+str(month_from)+"to"+str(year_to)+str(month_to)+"_"+money+"_"+foot+ ".csv"
    
    year_months = []

    for i in range(year_from,year_to+1):
        for j in range(month_from,month_to+1):
            temp = [i, j] 
            year_months.append(temp)
    
    
    all_data = []
    for year, month in year_months:
        date_from = datetime.datetime(year, month, 1)# +NY
        date_to = date_from + relativedelta(months=+1, day=1)# +NY

        ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
        month_data = oandaJsonToPythonList(ret)#取得したデータを格納

        all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)


    df.columns = ['Datetime',"hour_range",
                  "day",
                  'Volume', 'Open', 'High', 'Low', 'Close']
    df = df.set_index('Datetime')
    
    df.to_csv(filename, mode = 'w')
    
    return df

select_data()

,hour_range,day,Volume,Open,High,Low,Close
Datetime,,,,,,,
2010-01-03 18:00,18,3,35,92.943,93.000,92.925,92.934
2010-01-03 19:00,19,3,83,92.939,92.974,92.931,92.944
2010-01-03 20:00,20,3,97,92.942,92.990,92.930,92.952
2010-01-03 21:00,21,3,194,92.956,93.054,92.956,92.986
2010-01-03 22:00,22,3,933,92.984,93.090,92.888,92.930
...,...,...,...,...,...,...,...
2020-09-30 23:00,23,30,1726,105.418,105.458,105.418,105.453
2020-10-01 00:00,0,1,2855,105.452,105.547,105.441,105.504
2020-10-01 01:00,1,1,2602,105.502,105.551,105.478,105.514
